<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_dl_intro/4%20-%20Usando%20imagens%20do%20mundo%20real/20%20-%20Atividade%20Avaliativa/C1W4_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Laboratório Prático: Manuseio de imagens complexas - Conjunto de dados feliz ou triste (_Happy or Sad Dataset_)

Nesta tarefa, você usará o conjunto de dados feliz ou triste, que contém 80 imagens de rostos semelhantes a emojis, 40 felizes e 40 tristes.

Crie uma rede neural convolucional que seja treinada com 99,9% de acurácia nessas imagens e que cancele o treinamento ao atingir esse limite de acurácia de treinamento.

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os

## Carregar e explorar os dados

Comece dando uma olhada em algumas imagens do conjunto de dados.

Observe que todas as imagens estão contidas no diretório `./data/`. 

Esse diretório contém dois subdiretórios `happy/` e `sad/` e cada imagem é salva no subdiretório relacionado à classe a que pertence.

In [ ]:
! wget https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/cert_prof_dl_intro/4%20-%20Usando%20imagens%20do%20mundo%20real/20%20-%20Atividade%20Avaliativa/happy_sad.zip

In [ ]:
import zipfile

# Descompactar o conjunto de treinamento
local_zip = './happy_sad.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./happy-or-sad')

In [ ]:
from tensorflow.keras.preprocessing.image import load_img
import zipfile

# Descompactar o conjunto de treinamento
local_zip = './happy_sad.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./happy-or-sad')


base_dir = "./happy-or-sad"
happy_dir = os.path.join(base_dir, "happy/")
sad_dir = os.path.join(base_dir, "sad/")

print("Exemplo de imagem feliz")
plt.imshow(load_img(f"{os.path.join(happy_dir, os.listdir(happy_dir)[0])}"))
plt.show()

É legal poder ver exemplos das imagens para entender melhor o espaço do problema com o qual você está lidando. 

No entanto, ainda faltam algumas informações relevantes, como a resolução da imagem (embora o matplotlib renderize as imagens em uma grade, fornecendo uma boa ideia desses valores) e o valor máximo de pixel (isso é importante para normalizar esses valores).

Para isso, você pode usar o Keras, conforme mostrado na próxima célula:

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array

# Carregue o primeiro exemplo de um rosto feliz
sample_image  = load_img(f"{os.path.join(happy_dir, os.listdir(happy_dir)[0])}")

# Converta a imagem em sua representação de matriz numérica
sample_array = img_to_array(sample_image)

print(f"Cada imagem tem uma forma: {sample_array.shape}")

print(f"O valor máximo de pixel usado é: {np.max(sample_array)}")

Parece que as imagens têm uma resolução de 150x150.**Isso é muito importante porque esse será o tamanho da entrada da primeira camada da rede.**

**A última dimensão refere-se a cada um dos 3 canais RGB usados para representar imagens coloridas.**

## Definição da chamada de retorno

Como você já codificou o retorno de chamada responsável por interromper o treinamento (quando um nível desejado de acurácia é atingido) nas duas tarefas anteriores, desta vez ele já foi fornecido para que você possa se concentrar nas outras etapas:

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.999:
            print("\nAtingi 99,9% de acurácia, portanto, estou cancelando o treinamento!")
            self.model.stop_training = True

Uma observação rápida sobre callbacks: 

Até agora, você usou apenas a callback `on_epoch_end`, mas há muitas outras.

Por exemplo, talvez você queira dar uma olhada na chamada de retorno [EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping), que permite salvar os melhores pesos para o seu modelo.

## Pré-processamento dos dados

O Keras oferece excelente suporte para o pré-processamento de dados de imagem. Muito pode ser feito com o uso da classe `ImageDataGenerator`.

Não deixe de consultar o [docs](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) se você ficar preso no próximo exercício.

Em particular, talvez você queira prestar atenção ao argumento `rescale` ao instanciar o `ImageDataGenerator` e ao método [`flow_from_directory`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#flow_from_directory).

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def image_generator():
    ### INICIE SEU CÓDIGO AQUI

    # Instanciar a classe ImageDataGenerator.
    # Lembre-se de definir o argumento rescale.
    train_datagen = None

    # Especifique o método para carregar imagens de um diretório e passe os argumentos apropriados:
    # - directory: deve ser um caminho relativo para o diretório que contém os dados
    # targe_size: defina isso como igual à resolução de cada imagem (excluindo a dimensão da cor)
    # - batch_size: número de imagens que o gerador produz quando solicitado para um próximo lote. Defina esse valor como 10.
    # - class_mode: Como os rótulos são representados. Deve ser uma das opções "binary" (binário), "categorical" (categórico) ou "sparse" (esparso).
    # Escolha o que for mais adequado aqui, já que os rótulos serão rótulos binários 1D.
    train_generator = train_datagen.flow_from_directory(directory=None,
                                                        target_size=(None, None),
                                                        batch_size=None,
                                                        class_mode=None)
    ### TERMINE SEU CÓDIGO AQUI

    return train_generator
    

In [ ]:
# Salve seu gerador em uma variável
gen = image_generator()

**Resultado esperado:**
```
Encontradas 80 imagens pertencentes a 2 classes.
```

## Criação e treinamento do modelo

Por fim, conclua a função `train_happy_sad_model` abaixo. Essa função deve retornar sua rede neural.

**Seu modelo deve atingir uma acurácia de 99,9% ou mais antes de 15 épocas para ser aprovado nesta tarefa.**

**Dicas:**
- Você pode tentar qualquer arquitetura para a rede, mas tenha em mente que o modelo funcionará melhor com 3 camadas convolucionais.

In [ ]:
from tensorflow.keras import optimizers, losses

def train_happy_sad_model(train_generator):

    # Instanciar a chamada de retorno
    callbacks = myCallback()

    ### INICIE O CÓDIGO AQUI

    # Definir o modelo
    model = tf.keras.models.Sequential([
        None,
    ])

    # Compilar o modelo
    # Selecione uma função de perda compatível com a última camada de sua rede
    model.compile(loss=losses.None,
                  optimizer=optimizers.None,
                  metrics=['accuracy']) 
    


    # Treine o modelo
    # Seu modelo deve atingir a acurácia desejada em menos de 15 épocas.
    # Você pode programar até 20 épocas na função abaixo, mas a chamada de retorno deve ser acionada antes de 15.
    history = model.fit(x=None,
                        epochs=None,
                        callbacks=[None]
                       ) 
    
    ### TERMINE O CÓDIGO AQUI
    return history

In [ ]:
hist = train_happy_sad_model(gen)

Se a mensagem definida no retorno de chamada for impressa após menos de 15 épocas, isso significa que o retorno de chamada funcionou como esperado e o treinamento foi bem-sucedido.

Você também pode fazer uma verificação dupla executando a seguinte célula:

In [ ]:
print(f"Seu modelo atingiu a acurácia desejada após {len(hist.epoch)} épocas")

Se a sua chamada de retorno não interrompeu o treinamento, uma das causas pode ser o fato de você ter compilado o modelo usando uma métrica diferente de `accuracy` (como `acc`).

Certifique-se de que você definiu a métrica como `accuracy`. Você pode verificar executando a seguinte célula:

In [ ]:
if not "accuracy" in hist.model.metrics_names:
    print("Use a "acurácia" como métrica ao compilar seu modelo.")
else:
    print("A métrica foi definida corretamente.")

**Parabéns!**

Você implementou com sucesso uma CNN para ajudá-lo na tarefa de classificação de imagens complexas. Bom trabalho!

**Continue assim!**